# Step 4: Create matches by team DataFrame

When analyzing the data, we also like to have a dataframe `df_mbt (df_matches_by_team)` that contains, for each match, a separate row for each team participating in that match.
This structure is nicely visualized [at the Nufflytics blog](https://www.nufflytics.com/post/the-value-of-tv/).
Such a dataset is suitable for adding, at the match level, data that is specific for each team - coach pair, such as team value, coach rating etc.
For example, we can imagine adding more team level data, such as casualties caused during the match, or team composition at the start of the match etc.


In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
import h5py

from mizani.formatters import date_format

# point this to the location of the HDF5 datasets
path_to_datasets = 'datasets/current/'

# FUMBBL matches
target = 'df_matches.h5'
df_matches = pd.read_hdf(path_to_datasets + target) # takes huge memory now!!




In [ ]:
filename_hdf = path_to_datasets + target

def h5_tree(val, pre=''):
    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if type(val) == h5py._hl.group.Group:
                print(pre + '└── ' + key)
                h5_tree(val, pre+'    ')
            else:
                print(pre + '└── ' + key + ' (%d)' % len(val))
        else:
            if type(val) == h5py._hl.group.Group:
                print(pre + '├── ' + key)
                h5_tree(val, pre+'│   ')
            else:
                print(pre + '├── ' + key + ' (%d)' % len(val))

with h5py.File(filename_hdf, 'r') as hf:
    print(hf)
    h5_tree(hf)


In [ ]:
df_matches

In [ ]:
# make two copies, one for each team in the match
team1_data = df_matches[['match_id', 'match_date', 'week_number',	'year',	'week_year', 'week_date',
                          'team1_id', 'division_id', 'division_name', 'scheduler' , 'group_id', 'group_name',  'tournament_id' , 'tournament_name',
    'team1_coach_id', 'team1_race_name', 'team1_value', 'team1_score', 'team1_win', 'tv_diff', 'tv_match', 'team2_coach_id', 'team2_race_name', 'team2_value', 'team2_score', 
    'team1_comp', 'team1_pass', 'team1_rush', 'team1_block', 'team1_foul', 'team1_cas', 'team1_cas_bh', 'team1_cas_si', 'team1_cas_rip',
    'team2_comp', 'team2_pass', 'team2_rush', 'team2_block', 'team2_foul', 'team2_cas', 'team2_cas_bh', 'team2_cas_si', 'team2_cas_rip',
    'tv_bin', 'tv_bin2', 'mirror_match', 'coach1_CR', 'coach2_CR', 'coach1_CR_bin', 'CR_diff',  'has_sp']].copy()

team2_data = df_matches[['match_id', 'match_date', 'week_number',	'year',	'week_year', 'week_date', 
                         'team2_id',  'division_id', 'division_name', 'scheduler' , 'group_id', 'group_name',  'tournament_id' , 'tournament_name',
    'team2_coach_id', 'team2_race_name', 'team2_value', 'team2_score', 'team2_win', 'tv_diff', 'tv_match', 'team1_coach_id', 'team1_race_name', 'team1_value', 'team1_score',
    'team2_comp', 'team2_pass', 'team2_rush', 'team2_block', 'team2_foul', 'team2_cas', 'team2_cas_bh', 'team2_cas_si', 'team2_cas_rip',
    'team1_comp', 'team1_pass', 'team1_rush', 'team1_block', 'team1_foul', 'team1_cas', 'team1_cas_bh', 'team1_cas_si', 'team1_cas_rip',
    'tv_bin', 'tv_bin2', 'mirror_match', 'coach2_CR', 'coach1_CR', 'coach2_CR_bin','CR_diff', 'has_sp']].copy()

team1_data.columns = team2_data.columns = ['match_id', 'match_date', 'week_number',	'year',	'week_year', 'week_date', 
                                           'team_id', 'division_id', 'division_name', 'scheduler' , 'group_id', 'group_name',  'tournament_id' , 'tournament_name',
    'coach_id', 'race_name', 'team_value', 'team_score', 'wins', 'tv_diff', 'tv_match',  'away_coach_id', 'away_race_name', 'away_team_value', 'away_team_score',
    'home_comp', 'home_pass', 'home_rush', 'home_block', 'home_foul',  'home_cas', 'home_cas_bh', 'home_cas_si', 'home_cas_rip',
    'away_comp', 'away_pass', 'away_rush', 'away_block', 'away_foul',  'away_cas', 'away_cas_bh', 'away_cas_si', 'away_cas_rip',
    'tv_bin', 'tv_bin2', 'mirror_match', 'coach_CR', 'away_coach_CR', 'coach_CR_bin','CR_diff', 'has_sp']

# combine both dataframes
df_mbt = pd.concat([team1_data, team2_data])

df_mbt['tv_diff2'] = df_mbt['team_value'] - df_mbt['away_team_value']
df_mbt['cr_diff2'] = df_mbt['coach_CR'] - df_mbt['away_coach_CR']

## Adding outcome weights

One way to measure team strength is to calculate a win rate.
If we want to calculate win rates, we need to decide how to weigh a draw.
In Blood Bowl data analysis, it seems that a 2:1:0 (W / D / L) weighting scheme is most commonly used. 
So if we want to compare with others, it makes sense to adapt this scheme as well.
If we divide these weights by two we get something that, if we average it, we can interpret as a win rate.

This scheme has the advantage that the weighted average win percentage over all matches is always 50%, creating a nice reference point allowing conclusions such as "this and that team has an x percent above average win percentage"

In [ ]:
df_mbt.loc[df_mbt['wins'] == 0, 'wins'] = 0.5
df_mbt.loc[df_mbt['wins'] == -1, 'wins'] = 0

# convert to float
df_mbt['wins'] = df_mbt['wins'].astype(float)

At this point, Lets have a look at our dataset again:

In [ ]:
df_mbt.query("coach_id == 255851").sort_values('match_date')


# Step 5: adding race classifications to df_mbt (team tiers, bash/dash/hybrid)

There are team classifications for strength (tiers) and classifications that focus on play style (Bash/Dash/Hybrid).
The Bash/Dash/Hybrid classification i use is based on this great [Nufflytics blog](https://www.nufflytics.com/post/bash-dash-hybrid-by-the-numbers/), and my own subjective choices.
Here we add both to the `df_mbt` dataframe, as in this dataset each row is about a single team, instead of a pairing.

According to [this article from the NAF from 2017](https://www.thenaf.net/2017/05/tiers/), already since 2010 efforts were made to balance things out a bit between the different team strengths. For example, the weaker teams get more gold to spend on players, or get more so-called "Star player points" to spend on skilling players up. 

According to [the NAF](https://www.thenaf.net/tournaments/information/tiers-and-tiering/), traditionally team tiering consists of three groups, with Tier 1 being the strongest teams, and tier 3 the weakest teams. 

The GW BB2020 rule book also contains three tier groups, that are similar to the NAF tiers: except for Humans and Old World Alliance. 

And in november 2021, Games Workshop published an update of the three tier groups, now with High Elves moving from tier 2 to tier 1, and Old World Alliance moving back to tier 2.

Finally, I added an additional race classification into the well known bash/dash (agile)/hybrid / stunty classes.

**PM Vampires is on other**


In [ ]:
race_tiers = pd.read_csv('codelists/race_tiers_mapping.csv')
#race_tiers = race_tiers[ ['race_name', 'bb2020_tier', 'naf_tier', 'bb2020_nov21_tier', 'race_type']]
#race_tiers = race_tiers.dropna()

In [ ]:
race_tiers

In [ ]:
# add bb2020 tiers
df_mbt = pd.merge(df_mbt, race_tiers, on='race_name', how='left')



In [ ]:
target = 'datasets/current/df_mbt'

df_mbt.to_hdf(target + '.h5', key='df_mbt', mode='w', format = 't',  complevel = 9)
df_mbt.to_csv(target + '.csv')
